In [22]:
import os
import sagemaker
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.pytorch import PyTorchProcessor
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.pipeline import Pipeline

role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()

bucket = "lxeml"
prefix = "CH_Test"
base_path = f"s3://{bucket}/{prefix}/"

# -------------------------------------------------
# STEP 1: Preprocess Test Data
# -------------------------------------------------
sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    base_job_name="test-preprocess"
)

preprocess_test_step = ProcessingStep(
    name="PreprocessTestData",
    processor=sklearn_processor,
    code="preprocess_test_file_sagemaker.py",
    inputs=[
        ProcessingInput(
            source=os.path.join(base_path, "TEST.csv"),
            destination="/opt/ml/processing/input/test"
        ),
        ProcessingInput(
            source=os.path.join(base_path, "list_of_categories.csv"),
            destination="/opt/ml/processing/input/categories"
        ),
        # If needed for encoders:
        # ProcessingInput(
        #     source=os.path.join(base_path, "encoders"),
        #     destination="/opt/ml/processing/input/encoders"
        # )
    ],
    outputs=[
        ProcessingOutput(
            output_name="test_data",
            source="/opt/ml/processing/output",
            destination=os.path.join(base_path, "preprocessed_test")
        )
    ]
)

# -------------------------------------------------
# STEP 2: Predict
# -------------------------------------------------
model_s3_uri = "s3://lxeml/CH_Test/results/pipelines-f6evmcmrynvn-TrainModel-p0Z3pUjfMG/output/model.tar.gz"

predict_processor = PyTorchProcessor(
    framework_version="1.7.1",
    py_version="py3",
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    base_job_name="test-predict",
    env={"SAGEMAKER_PROGRAM": "predict.py"}
)

predict_step = ProcessingStep(
    name="PredictTestData",
    processor=predict_processor,
    code="predict.py",
    inputs=[
        ProcessingInput(
            source=preprocess_test_step.properties.ProcessingOutputConfig.Outputs["test_data"].S3Output.S3Uri,
            destination="/opt/ml/processing/input/test_data"
        ),
        ProcessingInput(
            source=model_s3_uri,
            destination="/opt/ml/processing/input/model"
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="predictions",
            source="/opt/ml/processing/output",
            destination=os.path.join(base_path, "test_predictions")
        )
    ]
)

# -------------------------------------------------
# BUILD & RUN the Pipeline
# -------------------------------------------------
test_inference_pipeline = Pipeline(
    name="CHTestInferencePipeline",
    steps=[preprocess_test_step, predict_step],
    sagemaker_session=sagemaker_session
)

test_inference_pipeline.upsert(role_arn=role)
print("Inference pipeline definition uploaded.")

execution = test_inference_pipeline.start()
execution.wait()
print("Inference pipeline execution complete.")

[03/29/25 22:11:27] INFO     Defaulting to only available Python version: py3                     ]8;id=51052;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=763992;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#610\610]8;;\

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=625506;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=664419;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

[03/29/25 22:11:28] WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=560615;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=912088;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

[03/29/25 22:11:29] WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=223261;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=418699;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=378319;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=823144;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=935854;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=219528;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

Inference pipeline definition uploaded.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:106                                                                                  │
│                                                                                                  │
│   103 print("Inference pipeline definition uploaded.")                                           │
│   104                                                                                            │
│   105 execution = test_inference_pipeline.start()                                                │
│ ❱ 106 execution.wait()                                                                           │
│   107 print("Inference pipeline execution complete.")                                            │
│   108                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline.py:934 in wait               │
│                                                                                                  │
│    931 │   │   waiter = botocore.waiter.create_waiter_with_client(                               │
│    932 │   │   │   waiter_id, model, self.sagemaker_session.sagemaker_client                     │
│    933 │   │   )                                                                                 │
│ ❱  934 │   │   waiter.wait(PipelineExecutionArn=self.arn)                                        │
│    935 │                                                                                         │
│    936 │   def result(self, step_name: str):                                                     │
│    937 │   │   """Retrieves the output of the provided step if it is a ``@step`` decorated func  │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/waiter.py:55 in wait                            │
│                                                                                                  │
│    52 │   # Waiter.wait method. This is needed to attach a docstring to the                      │
│    53 │   # method.                                                                              │
│    54 │   def wait(self, **kwargs):                                                              │
│ ❱  55 │   │   Waiter.wait(self, **kwargs)                                                        │
│    56 │                                                                                          │
│    57 │   wait.__doc__ = WaiterDocstring(                                                        │
│    58 │   │   waiter_name=waiter_name,                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/waiter.py:374 in wait                           │
│                                                                                                  │
│   371 │   │   │   │   return                                                                     │
│   372 │   │   │   if current_state == 'failure':                                                 │
│   373 │   │   │   │   reason = f'Waiter encountered a terminal failure state: {acceptor.explan   │
│ ❱ 374 │   │   │   │   raise WaiterError(                                                         │
│   375 │   │   │   │   │   name=self.name,                                                        │
│   376 │   │   │   │   │   reason=reason,                                                         │
│   377 │   │   │   │   │   last_response=response,                                                │
╰────────────────────────────────────────────────────────────